# Correlation energy


The influence of electron correlation on structural parameters such as
bond lengths and angles is not negligible. Hartree-Fock calculations may
give bond lengths that deviate considerably from the experimental value.
Comparably cheap methods such as MPn may often remedy sufficiently for
this problem, but some smaller deviations may remain. The most rigorous
approach in such cases would be *e.g.* a full configuration interaction
(full CI/FCI), but this is computationally untractable for any larger
molecule using a reasonable basis. One may therefore restrict the method
to singly and doubly excited Slater determinants, assuming that the main
effects of correlation are captured well enough in such a truncation.
However, such truncations introduce size-consistency problems.

 ![image](../../images/HNO3.png)

## Influence of correlation on geometry

```{admonition} Exercise
:class: exercise 

Assess and comment on the performance of HF, MP2 and MP3 with respect to the prediction of bond lengths and bond angles in HNO$_3$ in reference to the experimental values [^no3]
```

[^no3]: Anglada J. M., Olivella, S., Sole A., Phys. Chem. Chem. Phys., 2014 16, 19437

In [1]:
import psi4
import py3Dmol
import pandas as pd
import numpy as np

import sys
sys.path.append("..")
from helpers import *

In [2]:
hno3 = psi4.geometry("""
0 1
symmetry c1
O           1.326  -0.373  -0.127
N           0.015  -0.015   0.415
O          -0.329   1.292  -0.145
O          -0.944  -0.973  -0.135
H          -1.811  -0.738   0.223""")

In [3]:
drawXYZ(hno3)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
psi4.set_options({'reference':'uhf','guess':'gwh'})

In [5]:
hno3_hf =  hno3.clone() # we clone the molecule as we want to start the geometry optimization from the same starting structure

In [6]:
E_hf, history_hf = psi4.optimize('hf/6-31+G*', molecule=hno3_hf, return_history=True)

Optimizer: Optimization complete!


In [5]:
# perform the calculations for mp2 and mp3 here also creating cloned molecules
# also create a history file as we will use it to get the last optimized geometries from psi4

In [8]:
#save last optimized coordinates in a dictionary
opt_coord = {
    'hf':np.array(history_hf['coordinates'][-1]),
    'mp2':np.array(history_mp2['coordinates'][-1]),
    'mp3':np.array(history_mp3['coordinates'][-1])
}


:::{margin}
the distance between two points is the L2 norm (Euclidean distance)
```
a = np.array([0,0,0])
b = np.array([1,1,1])
np.linalg.norm(a-b)
```
this has been implemented in the `calculate_bond` function contained in `helpers.py` imported above. 
:::

The calculation for the angle is also possible using numpy. You can use the following function

In [43]:
def calculate_angle(a,b,c):
    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

In [9]:
drawXYZ_labeled(hno3)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [46]:
# we use the opt_coord dictionary to cacluate the properties using python 

# the units are in bohr (atomic unit of distance), we convert to Å 

results = {'hf':{},'mp2':{},'mp3':{},'exp':{'r(O1-N) Å':1.198, 'r(O2-N) Å':1.410, 'r(O3-N) Å':1.213, 'φ(O-N-O) °':130.2}, }
for method in ['hf', 'mp2', 'mp3']:
    xyz = opt_coord[method] # store the geometry array optimized with the current method in a temp variable
    results[method]['r(O1-N) Å'] = calculate_bond(xyz[0], xyz[1]) * psi4.constants.bohr2angstroms
    results[method]['r(O2-N) Å'] = calculate_bond(xyz[2], xyz[1]) * psi4.constants.bohr2angstroms
    results[method]['r(O3-N) Å'] = calculate_bond(xyz[3], xyz[1]) * psi4.constants.bohr2angstroms
    results[method]['φ(O-N-O) °'] = calculate_angle(xyz[0], xyz[1], xyz[method][2])

In [ ]:
pd.DataFrame.from_dict(results)

## Recovering Correlation Energy

:::{margin} Attribution
This section was adapted from Shepherd et. al. under BSD-3 license. 
:::

At the Hartree Fock level of theory, each electron experiences an average potential field of all the other electrons. In essence, it is a "mean field" approach that neglects individual electron-electron interactions or "electron correlation". Thus, we define the difference between the self-consistent field energy and the exact energy as the correlation energy. Two fundamentally different approaches to account for electron correlation effects are available by selecting a Correlation method: Moller Plesset (MP) Perturbation theory and Coupled Cluster (CC) theory. 




 

```{admonition} Question 5
:class: exercise 

Calculate the SCF energy for Boron using the *6-311+G** basis set, then determine the value of the correlation energy for boron assuming an "experimental" energy of **-24.608 hartrees** [^schaefer]


Using the same basis set, perform an energy calculation with MP2 and MP4.  

Using the same basis set, perform an energy calculation with CCSD and CCSD(T).

Determine the percentage of the correlation energy recovered for HF, MP2, MP4, CCSD, CCSD(T). 
```

```{admonition} CCSD energy/mp2 energy
:class: tip
You may recover the CCSD energy from the CCSD(T) calculation but you will have to look at the output file or by calculating both. The calculations for a single atom are quick.  

The same is true for the MP2 energy which can be extracted from the MP4 calculation.
MP4 will require the use of the following options: `psi4.set_options({"reference" :"rohf", "qc_module":"detci"})` as psi4 does not have MP4 implemented using a rhf reference. 
```


[^schaefer]: H. S. Schaefer and F. Harris, (1968) Phys Rev. 167, 67


In [60]:
psi4.core.clean_options()

# define a single boron atom and set the correct multiplicity
boron_geometry = psi4.geometry("""""")

psi4.set_options({'reference' : "UHF"})

# Perform single-point calculation with requested method/basis 
B_energy_hf  = 0

In [ ]:
#Experimental energy of Boron
B_exact_e = -24.608


correlation = None # Calculate the total amount of correlation energy recovered 

print(F"\n Electron correlation: {correlation} hartrees")

In [20]:
# calculate energy for other methods
#MP2
e_mp2 = None
#CCSD
e_ccsd = None 
#CCSD(T)
e_ccsd_t = None
#MP4
psi4.set_options({"reference" :"rohf", 
                   "qc_module":"detci"})
e_mp4 = None 

In [21]:
correlations = {}
correlations['hf']= 0
correlations['mp2']    = None # calculate the % of correlation energy recovered here
correlations['mp4']    = None
correlations['ccsd']   = None
correlations['ccsd(t)'] = None

In [ ]:
pd.DataFrame.from_dict(correlations,orient='index',columns=['% correlation recovered'])